In [2]:
import os
import cv2
import imutils
import numpy as np
from matplotlib import pyplot as plt
from classifier_jigsaw import Classifier
from helper import get_biggest_square_corners, get_structure_matrix
from helper import apply_filter_to_images, crop_the_images
from helper import break_the_images_in_81_tiles, rotate_the_images_to_upright

In [3]:
def read_images(path_to_images, number_of_images):
    images = list()

    
    for i in range(0, number_of_images):

        aux_file_name = ""
        if i < 9:
            aux_file_path = path_to_images + '0' + str(i+1) + '.jpg'
        else:
            aux_file_path = path_to_images + str(i+1) + '.jpg'

        aux_image = cv2.imread(aux_file_path)
        images.append(aux_image)


    return images


In [4]:
def detect_strong_lines_on_vertical(image):
    width, height = image.shape[1] , image.shape[0]
    list_of_white_pixels = list()
    vertical_line_types = list()
    for line in range(9):
        for col in range(8): 
            aux = image[height//9 * line + 50: height//9 * (line+1) - 50, width//9 * col + width//9//2: width//9 * (col+1) + width//9//2]
            auxWidth, auxHeight = aux.shape[1], aux.shape[0]
            aux = aux[auxHeight//4: auxHeight//2, auxWidth//4 + 20: auxWidth//4*3 - 30]
            sum = 0
            for i in range(len(aux)):
                for j in range(len(aux[i])):
                    if aux[i][j] > 100:
                        sum += 1
            list_of_white_pixels.append(sum)
    
    suma_totala = 0
    for pixel in list_of_white_pixels:
        suma_totala += pixel
    mean = suma_totala//72

    for line in range(9):
        curr_line = list()
        for col in range(8): 
            aux = image[height//9 * line + 50: height//9 * (line+1) - 50, width//9 * col + width//9//2: width//9 * (col+1) + width//9//2]
            auxWidth, auxHeight = aux.shape[1], aux.shape[0]
            aux = aux[auxHeight//4: auxHeight//2, auxWidth//4 + 20: auxWidth//4*3 - 30]
            sum = 0
            for i in range(len(aux)):
                for j in range(len(aux[i])):
                    if aux[i][j] > 100:
                        sum += 1
            line_type = 0
            if sum > mean:
                line_type = 1

            curr_line.append(line_type)
        vertical_line_types.append(curr_line)
            
    return vertical_line_types

In [5]:
def detect_strong_lines_on_horizontal(image):
    list_of_white_pixels = list()
    horizontal_line_types = list()
    width, height = image.shape[1] , image.shape[0]
    for line in range(8):
        for col in range(9): 
            aux = image[height//9 * line + height//9//2 + height//9//3 + 30: height//9 * (line+1) + height//9//2 - height//9//3, width//9 * col + 90: width//9 * (col+1) - 90]
            sum = 0
            for i in range(len(aux)):
                for j in range(len(aux[i])):
                    if aux[i][j] > 100:
                        sum += 1
            list_of_white_pixels.append(sum)

    
    suma_totala = 0
    for pixel in list_of_white_pixels:
        suma_totala += pixel
    mean = suma_totala//72

    for line in range(8):
        curr_line = list()
        for col in range(9): 
            aux = image[height//9 * line + height//9//2 + height//9//3 + 30: height//9 * (line+1) + height//9//2 - height//9//3, width//9 * col + 90: width//9 * (col+1) - 90]
            sum = 0
            for i in range(len(aux)):
                for j in range(len(aux[i])):
                    if aux[i][j] > 100:
                        sum += 1
            line_type = 0
            if sum > mean:
                line_type = 1
            

            curr_line.append(line_type)
        horizontal_line_types.append(curr_line)
            
    return horizontal_line_types

In [6]:
def change_structure_of_structureMatrix(structure_matrixes):
    return_structure_matrixes = list()
    for structure_matrix in structure_matrixes:
        return_structure_matrix = list()
        index = 0
        curr_line = list()
        for each_pair in structure_matrix:
            for each_element in each_pair:
                curr_line.append(each_element)
            index += 1
            if index == 3:
                index = 0
                return_structure_matrix.append(curr_line)
                curr_line = list()
        return_structure_matrixes.append(return_structure_matrix)

    return return_structure_matrixes


In [7]:
def recursive_mark_squares(zone_matrix, curr_number, index_line, index_col, vertical_strong_lines, horizontal_strong_lines):
    zone_matrix[index_line][index_col] = curr_number

    if index_line != 0:
        if zone_matrix[index_line - 1][index_col] == 0 and horizontal_strong_lines[index_line - 1][index_col] == 0:
            recursive_mark_squares(zone_matrix, curr_number, index_line-1, index_col, vertical_strong_lines, horizontal_strong_lines)

    if index_line != 8:
        if zone_matrix[index_line + 1][index_col] == 0 and horizontal_strong_lines[index_line][index_col] == 0:
            recursive_mark_squares(zone_matrix, curr_number, index_line + 1, index_col, vertical_strong_lines, horizontal_strong_lines)

    if index_col != 0:
        if zone_matrix[index_line][index_col - 1] == 0 and vertical_strong_lines[index_line][index_col - 1] == 0:
            recursive_mark_squares(zone_matrix, curr_number, index_line, index_col - 1, vertical_strong_lines, horizontal_strong_lines)
    
    if index_col != 8:
        if zone_matrix[index_line][index_col + 1] == 0 and vertical_strong_lines[index_line][index_col ] == 0:
            recursive_mark_squares(zone_matrix, curr_number, index_line, index_col + 1, vertical_strong_lines, horizontal_strong_lines)


In [8]:
def create_zone_matrix(vertical_strong_lines, horizontal_strong_lines):
    zone_matrix = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
    curr_number = 1
    
    for index_line in range(9):
        for index_col in range(9):
            if zone_matrix[index_line][index_col] == 0:
                recursive_mark_squares(zone_matrix, curr_number, index_line, index_col, vertical_strong_lines, horizontal_strong_lines)
                curr_number += 1
    
    return zone_matrix

In [9]:
def join_structure_and_zone_matrixes(structure_matrix, zone_matrix):
    """
    Joins the structure matrix and zone matrix in order to create the result for task 2.

    Args:
        structure_matrix
        zone_matrix

    Returns:
        final_result
    """
    final_result = list()
    for i in range(9):
        line = list()
        for j in range(9):
            line.append(zone_matrix[i][j])
            line.append(structure_matrix[i][j])
        final_result.append(line)
    return final_result

In [10]:
if __name__ == "__main__":
    # Just reading the images.
    number_of_images = 40
    path_to_images = './Tema1/antrenare/jigsaw/'
    jigsaw_sudoku_raw_images = read_images(path_to_images = path_to_images , number_of_images = number_of_images)

    # Prelucrating the images. Applying color change to gray, some Guasssian blur (0) and otsu treshold.
    jigsaw_sudoku_prelucrated_images = apply_filter_to_images(images = jigsaw_sudoku_raw_images, 
                                    to_gray = True, blur_level = 3, sharpen = False, treshold = 'otsu')
    cv2.imwrite('task1_jig.png', jigsaw_sudoku_prelucrated_images[0])
    input()

    # Using the corners of the contour of the biggest square in each image, I rotate the image
    #   until the upper corners are as alligned as possible. This should be enough to solve the tilted images problems.                            
    rotated_sudoku_prelucrated_images = rotate_the_images_to_upright(images = jigsaw_sudoku_prelucrated_images)
    
    # Again, using the corners of the contour of the biggest square in each image, I make a cut in the image,
    #   keeping only the area between the corners, in order to keep only the sudoku, 
    #   which is the part that I am interested in the most.
    #       offset - how many more pixels you want to delete from each margin. 
    #                   Using it to get rid (as much as possible) of the lines around the big sudoku square.
    cropped_sudoku_images = crop_the_images(images = rotated_sudoku_prelucrated_images)

    # If all past operations went well, here I break the image in 81 tiles. Each tile will represent 1/81 of the 
    #   image and it will corespond (if everything before went well) to a small square from the sudoku. 
    #       offset - how many more pixels you want to delete from each margin. 
    #                   Using it to get rid (as much as possible) of the lines around each small square.
    breaked_sudoku_images = break_the_images_in_81_tiles(images=cropped_sudoku_images, offset=60)

    # This function gets a list of small squares images and tries to see which ones have something in them
    #   and which not, based on how many white pixels each small square has inside of it. 
    structure_matrixes = list(get_structure_matrix(breaked_image, threshold = 600) for breaked_image in breaked_sudoku_images)
    structure_matrixes = change_structure_of_structureMatrix(structure_matrixes=structure_matrixes)

    # This method runs though all lines that appear vertically on each line of the sudoku and checks whether a line
    #   is thick or not (separetes two different zones). it saves the one that separetes two zones as 1 and ones who not as 0.
    vertical_strong_lines = list(detect_strong_lines_on_vertical(cropped_sudoku_image) for cropped_sudoku_image in cropped_sudoku_images)
    # This method runs though all lines that appear horizontally on each collumn of the sudoku and checks whether a line
    #   is thick or not (separetes two different zones). it saves the one that separetes two zones as 1 and ones who not as 0.
    horizontal_strong_lines = list(detect_strong_lines_on_horizontal(cropped_sudoku_image) for cropped_sudoku_image in cropped_sudoku_images)

    # Here I am creating the zone matrixes, basically how the square is divided by the thick lines.
    zone_matrixes = list()
    for index in range(number_of_images):
        zone_matrixes.append(create_zone_matrix(vertical_strong_lines[index], horizontal_strong_lines[index]))
    
    # =========================== TASK 2 - Extract each sudoku structure and zones ===============================
    # Info: The result should be a matrix which contains 'o' for each empty small square in the sudoku 
    #           and 'x' for each occupied (by a number) small square in the sudoku.
    #            Also, each zone needs to be pointed out in the structure matrix.

    # Now that I know how the sudoku is divided in zone, I only need to join the structure matrix (the ones
    #   which tells me if a square contains a number or not) and the zone_matrix to get the results for TASK 2
    for index in range(number_of_images):
        final_output = str()
        joined = join_structure_and_zone_matrixes(structure_matrixes[index], zone_matrixes[index])
        ind = 0
        for line in joined:
            for element in line:
                final_output += str(element)
            ind += 1
            if ind != 9:
                final_output += '\n'

        with open("./fisiere_solutie/Gherasim_Rares_343/jigsaw/" + str(index+1) + "_predicted.txt", "w") as f:
            for i in range(len(final_output)):
                f.write(final_output[i])

    
    # ================== TASK 2 - BONUS - Extract each sudoku with numbers ===============================
    # Info: The result should be a matrix which contains 'o' for each empty small square in the sudoku 
    #           and the coresponding number for each ocupied small square in the sudoku.
    #            Also, each zone needs to be pointed out in the structure matrix.

    # This is a basic classifier that trains itself when initialized.
    # Basic svm.svc classifier with 'rbf' kernel and c set to 4.
    # The training images/labels are small tiles extracted from the 'jigsaw/antrenare' images and then
    #   classified by myself with numbers ranging from 1 to 9.
    classifier = Classifier()
    for index in range(number_of_images):
        final_output = str()
        ind = 0
        for i in range(len(breaked_sudoku_images[index])):
            for j in range(len(breaked_sudoku_images[index][i])):
                final_output += str(zone_matrixes[index][i][j])
                if structure_matrixes[index][i][j] == 'o':
                    final_output += 'o'
                else:
                    try:
                        prediction = classifier.predict(breaked_sudoku_images[index][i][j])
                    except:
                        print(i, ' ', j)
                        input()
                    final_output += str(prediction[0])
            ind += 1
            if ind != 9:
                final_output += '\n'

        with open("./fisiere_solutie/Gherasim_Rares_343/jigsaw/" + str(index+1) + "_bonus_predicted.txt", "w") as f:
                for i in range(len(final_output)):
                    f.write(final_output[i])

KeyboardInterrupt: Interrupted by user